# Instructions {-}

1. Download the provided jupyter notebook file to your computer.
2. Write all your answers and code into this notebook file.
3. When your work is completed, export your notebook to an HTML file.
4. Submit your HTML file and a copy of the notebook to the assignment page on Moodle.

__Please ensure you do not save your name as part of your file submission. Also, aim to suppress warning or error messages that may show information about your file path directory which could reveal your identity.__


__Please cite any use of AI and include the prompts you provided at the end of the document. The class policy allows for AI to assist in coding, but all written responses should be developed on your own.__

# Background Material on the Statistical Techniques {-}

This problem set will focus primarily on the LASSO and logistic regressions as new tools for prediction, but it will also draw upon concepts of MSE and prediction error within the testing and training samples. You will also be asked to calculate measures of precision and recall which are introduced in the case study and in lecture this / next week.

For a theoretical description of how the LASSO alters traditional regression approaches you should consult Chapter 6, section 6.2.2 of the course textbook (Introduction to Statistical Learning).

Another treatment of the LASSO and ridge regressions is provided in the course reading from Géron, Aurélien - Hands-on Machine Learning. Chapter 4 of this text contains a subsection on "Regularized Linear Models." You may find it helpful to read this brief subsection.

Other readings from these next weeks are strictly background and show either additional theoretical derivations behind these concepts or applications of these penalized regressions in a public policy context.

# Problem 1 - Introduction to the Case {-}

The plan for this case study is to have you learn and apply machine learning tools to make predictions in this problem set which will be due on __Tuesday December 2__. During our regular classes that week we will have a debrief session on this case study to discuss your results and think about some of the broader policy uses of prediction. Please have available a copy of your answers to these questions for this in class discussion.

You will do the empirical work related to the Occupational Safety and Health Administration (OSHA) case. To do this, you have been given an OSHA data set on injuries and inspections in work establishments. The goal is to help OSHA improve the way it selects establishments to be inspected. They are under increasing pressure because some policymakers are arguing that OSHA inspections frequently represent an unnecessary cost to businesses. In this problem set, you will be asked to use various algorithms designed to predict a key outcome variable that OSHA could take into account in deciding which establishments to select for inspections.

## 1-A {-}

Read the provided case study "Improving Worker Safety in the Era of Machine Learning” (HBS Case #N9-618-09). This case study is available via the [reading list](https://moodle.lse.ac.uk/mod/lti/view.php?id=1809514) on the course Moodle page under the Autumn Term: Week 8 Section.
 

## 1-B {-}

The case study outlines 4 potential approaches OSHA could select regarding how to identify workplaces to visit. After reading the case study, summarise the key advantages and disadvantages of each of the four targeting approaches compared to OSHA’s current approach, which amounts to selecting randomly from the list of sites with high historical injury rates. [1-2 paragraphs.]

_Your summary here:_

Each of the four OSHA site selection approaches has distinct strengths and limitations. The first approach is unique in relying on local knowledge and networks, enabling access to information that might otherwise remain unavailable or undisclosed. However, it is entirely discretionary, lacking standardized criteria for site selection. The third approach enables predictive analysis, though it remains vulnerable to the inherent flaws and biases of the forecasting model employed. The fourth approach combines historical data with predictive modeling, potentially mitigating the limitations of either method used independently, yet it still inherits weaknesses from both and introduces an element of randomness without clear justification for this design choice.

The second approach bears similarity to the current random selection process in that both rely on historical data and provide some degree of certainty regarding site risk levels. However, both suffer from dependence on outdated information, and their scope remains constrained by budget limitations. A key distinction exists: while the second method systematically selects locations with the highest historical risk indicators, the current method introduces randomization into the selection process, potentially reducing bias but also reducing targeting efficiency.

# Problem 2 - Data Description and Focus {-}

The provided `osha.csv` dataset contains annual observations for a sample of establishments that were in OSHA’s target list of potential sites to inspect through the Site-Specific Targeting (SST) program in the period 2001-2010. These tend to be establishments that are in higher-risk industries that merit OSHA’s close attention. Because of resource constraints, OSHA cannot inspect all sites on such a list. At the moment, the process that OSHA uses to select from this list is akin to a simple random selection. Your job is to see if you can use prediction models to improve on the procedure that OSHA uses.

__NOTE__: The case study prompt refers to an example with 6314 observations. I have provided you with a smaller random subset of this data set, so your descriptive statistics will not perfectly match those included in Exhibit 2a of the case appendix.

__Outcomes__

OSHA wants to predict two outcomes:

* `injury_rate`: a continuous variable, measured as a “workplace’s annual number of injuries that prompted at least one day away from work (DAFW) per 100 full time equivalent (FTE) employees”
* `high_injury_rate`: a binary variable, which is 1 if `injury_rate` is larger than 3 and 0 otherwise.

__Predictors__

All variables are described in the case (data appendix). Each observation in the data set is a workplace at risk for Site-Specific Targeting (SST) inspection. Some variables represent outcomes in the prediction year, some represent characteristics that are available before the prediction year, and some represent characteristics that are fixed over time (industry, region of the country, etc.).

__Predictive Models__

I will ask you to evaluate at least three models:

1. A simple Ordinary Least Squares (OLS) regression considering a small subset of predictors (referred to here as the `short regression`)
2. A "kitchen-sink", exhaustive OLS regression which considers all the predictors available (referred to here as the `long regression`)
3. A `LASSO regression` which also considers all the predictors available but runs a penalized regression

In later portions of the problem set, you will also examine the logistic regression equivalent of the above approaches.

## 2-A: Examining the Data and Outcome Variables {-}

Before beginning your analysis, briefly examine the structure and format of the data set.

There are too many individual variables to examine in detail, but you should at least examine and describe the key outcome variables identified above. Briefly summarise the data (1-2 paragraphs).

In [22]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
osha = pd.read_csv('(...).W8-9 osha_2025.csv')

In [21]:
# Your exploratory analysis here:

print(osha.head(10))
print(osha.info())
print(osha.columns)
print(osha.injury_rate.describe())
print(osha.high_injury_rate.describe())

   injury_rate  high_injury_rate  injuries  sst_year  num_yrs_prev_on_sst  \
0     3.473055                 1        11      2005            -0.205447   
1     4.882524                 1         5      2003            -0.821400   
2     5.110077                 1         6      2004            -0.821400   
3     3.028965                 1         7      2004            -0.821400   
4     3.740482                 1         3      2006            -0.821400   
5     4.642092                 1         6      2005             1.026457   
6     2.582741                 0         1      2007            -0.205447   
7     1.389168                 0         1      2006             0.410505   
8     4.922267                 1        41      2008             2.258361   
9     3.339488                 1         5      2006            -0.205447   

   sic_sector_11  sic_sector_21  sic_sector_31  sic_sector_42  sic_sector_44  \
0              0              0              0              0           

_Data summary:_

The OSHA dataset contains 2,149 observations across 183 variables. The majority of variables are continuous (140 float columns), with 43 integer columns. Key variables include `injury_rate` (mean: 3.40, range: 0.00–16.62) and `high_injury_rate`, a binary indicator where approximately 44% of establishments are classified as high-risk (value = 1). The dataset also includes temporal information (`sst_year`), industry classification (`sic_sector_*`), and various lagged injury metrics (prefixed with `l2_`), in accordance with the fact that the data tracks workplace safety outcomes over time, and collects historical injury patterns.

# Problem 3 - Preprocessing the Data and Creating Initial Models {-}

The data has been pre-processed in the following ways to make things easier to work with:

1. Categorical variables have been transformed into multiple dummy variables
2. Non-binary variables have been re-scaled so that each of them has a mean of 0 and a standard deviation of 1.[^1]

__NOTE__: For purposes of this assignment, there is no need to standardize the dummy variables further (although we will introduce techniques for this in the WT).
 
[^1]: This "standardizing" process helps make coefficients easier to compare across variables and will be necessary for a penalized regression like the LASSO to correctly penalize estimates irrespective of the unit of measurement.

## 3-A: Formula for the Short Regression {-}

The data appendix of the case study describes the variables that should be included in the short regression (in the column "Included in simple model" of the Data Appendix). A defensible choice for the short OLS is to start from indicators that the site has gone through inspections or has received some complaints in the past.

We can create a formula object to run with our regressions with the following code that includes all of the variables identified for inclusion in this regression.

```python
f_short_c = 'injury_rate ~ has_tmin1_odi + any_insp_prior + \
  any_complaint_tmin13 + num_nonfat_comp_insp_cy_tc99mm1 + \
  initial_pen_cy_mzmm1 + ln_initial_pen_cy_mzmm1 + \
  dafw_analysis_rec_tc99mm1'
```

In this code `f_short_c` will be a string representing our regression formula, where it is named for the "short" regression and will be used to predict the continuous outcome measure `injury_rate`. This entire string `f_short_c` can be passed as the argument in a regression using the formula notation.

In [232]:
# Writing the regressions formula
f_short_c = 'injury_rate ~ has_tmin1_odi + any_insp_prior + \
  any_complaint_tmin13 + num_nonfat_comp_insp_cy_tc99mm1 + \
  initial_pen_cy_mzmm1 + ln_initial_pen_cy_mzmm1 + \
  dafw_analysis_rec_tc99mm1'

Alternatively, we could save the desired columns as a matrix. Fitting models with this type of matrix is common in the machine learning literature and it can also be used using the `statsmodels.api` syntax. For your convenience, the below block of code saves these variables in a list should you wish to create this matrix.

In [233]:
short_vars = ['has_tmin1_odi', 'any_insp_prior', 'any_complaint_tmin13', 'num_nonfat_comp_insp_cy_tc99mm1',
                        'initial_pen_cy_mzmm1', 'ln_initial_pen_cy_mzmm1', 'dafw_analysis_rec_tc99mm1']

Using these provided variables and whichever technique you prefer, fit a simple OLS regression to the full data set using only the short model subset of variables (be sure your model also includes the intercept term). Doing this now will confirm your regression setup is correct before moving on to other tasks.

In [234]:
# Fit the short regression model to the full data

model = smf.ols(f_short_c, data=osha).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            injury_rate   R-squared:                       0.317
Model:                            OLS   Adj. R-squared:                  0.314
Method:                 Least Squares   F-statistic:                     141.7
Date:                Mon, 01 Dec 2025   Prob (F-statistic):          6.95e-172
Time:                        22:03:11   Log-Likelihood:                -5191.3
No. Observations:                2149   AIC:                         1.040e+04
Df Residuals:                    2141   BIC:                         1.044e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

## 3-B: Formula for the Long Regression {-}

The kitchen-sink, or "long" regression should include all the available variables in the data __EXCEPT__ for the following: `sst_year` (the year), `estab_id_duns` (the workplace identifier), `injuries`, `injury_rate`, and `high_injury_rate` (the three outcome measures). 

Estimate this "long" regression model on the full data set, again predicting the outcome `injury_rate`, but including all other variables except those noted above.

This formula will include many terms (178 features), and would be too long to type out by hand which would be prone to error.

Hint 1: using matrix notation for fitting the model avoids the need to create your own formula if you can drop the excluded columns.

Hint 2: if you want to create a regression formula, you can use the [str.join()](https://www.w3schools.com/python/ref_string_join.asp) method to reliably and quickly write the syntax.

In [235]:
# Fit the long regression model to the full data

cols_excluded = ['sst_year', 'estab_id_duns', 'injuries', 'injury_rate', 'high_injury_rate']
cols_used = [col for col in osha.columns if col not in cols_excluded]
formula = 'injury_rate ~ ' + '+'.join(cols_used)
kls_model = smf.ols(formula, data=osha).fit()
print(ks_model.summary())

                            OLS Regression Results                            
Dep. Variable:            injury_rate   R-squared:                       0.446
Model:                            OLS   Adj. R-squared:                  0.396
Method:                 Least Squares   F-statistic:                     8.896
Date:                Mon, 01 Dec 2025   Prob (F-statistic):          4.41e-153
Time:                        22:03:14   Log-Likelihood:                -4966.5
No. Observations:                2149   AIC:                         1.029e+04
Df Residuals:                    1970   BIC:                         1.131e+04
Df Model:                         178                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

There is no need to report your regression results or examine the coefficients in detail here, but do run the code to confirm there are no errors in this construction of your formulas before moving on to the next portions of the problem.

## 3-C: Splitting the Data {-}

Split the data 70-30 into training and test data sets respectively. Use the [train_test_split](https://scikit-learn.org/dev/modules/generated/sklearn.model_selection.train_test_split.html) function from `sklearn` and set the random_state argument to `407` to ensure your split matches the solutions. 

Briefly examine the structure of your training and testing splits. Report how many observations are in each sample.

In [236]:
# Split the data and examine the structure

from sklearn.model_selection import train_test_split
osha_train, osha_test = train_test_split(osha, test_size=.30, train_size=.70, random_state=407)
print(osha_train, osha_test)

      injury_rate  high_injury_rate  injuries  sst_year  num_yrs_prev_on_sst  \
1716     3.072996                 1         2      2001            -0.821400   
1339     7.193075                 1         6      2009            -0.821400   
102      1.168811                 0         1      2009             0.410505   
867      0.000000                 0         0      2006             0.410505   
783      6.068804                 1         5      2005             1.026457   
...           ...               ...       ...       ...                  ...   
212      0.000000                 0         0      2004            -0.205447   
93       1.044817                 0         1      2010             1.026457   
975      0.000000                 0         0      2010            -0.821400   
248      5.342503                 1         8      2001            -0.821400   
1755     0.000000                 0         0      2010            -0.821400   

      sic_sector_11  sic_sector_21  sic

In [237]:
# Structure of splits

print(f"Size of training set: {osha_train.shape[0]} observations, {osha_train.shape[1]} features")
print(f"Size of test set: {osha_test.shape[0]} observations, {osha_test.shape[1]} features")

Size of training set: 1504 observations, 183 features
Size of test set: 645 observations, 183 features


Throughout this problem it will be useful to have separate arrays saved containing our outcome variables for the `injury_rate` measure. Create new variables storing these measures from both the training and testing data. You can reference these variables both in your calls to fit models and when calculating measures of model fit like the MSE.

1. Save an array of the `injury_rate` in the training sample as `y_c_train`
2. Save an array of the `injury_rate` in the testing sample as `y_c_test`

Where the letter 'c' in the above variable names refers to this outcome measure being continuous.

In [238]:
# Store the outcome variable from the training / testing data

y_c_train = osha_train['injury_rate']
y_c_test = osha_test['injury_rate']

## 3-D: Linear Models {-}

You previously built both the "short" and "long" linear regression models for the full data set. Now, fit each of these models on the _training data_ you created above. Be sure to save your model objects as we will need to extract predictions and goodness of fit measures from these models.

In [239]:
# Fitting the short regression model

short_reg = smf.ols('y_c_train  ~ has_tmin1_odi + any_insp_prior + \
  any_complaint_tmin13 + num_nonfat_comp_insp_cy_tc99mm1 + \
  initial_pen_cy_mzmm1 + ln_initial_pen_cy_mzmm1 + \
  dafw_analysis_rec_tc99mm1', data=osha_train).fit()
print(short_reg.summary())

                            OLS Regression Results                            
Dep. Variable:              y_c_train   R-squared:                       0.332
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     106.2
Date:                Mon, 01 Dec 2025   Prob (F-statistic):          2.37e-126
Time:                        22:03:44   Log-Likelihood:                -3606.0
No. Observations:                1504   AIC:                             7228.
Df Residuals:                    1496   BIC:                             7271.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

In [240]:
# Fitting the long regression model

formula_lr = 'y_c_train ~ ' + '+'.join(cols_used)
long_reg = smf.ols(formula_lr, data=osha_train).fit()
print(long_reg.summary())

                            OLS Regression Results                            
Dep. Variable:              y_c_train   R-squared:                       0.475
Model:                            OLS   Adj. R-squared:                  0.404
Method:                 Least Squares   F-statistic:                     6.731
Date:                Mon, 01 Dec 2025   Prob (F-statistic):           5.56e-98
Time:                        22:03:57   Log-Likelihood:                -3425.1
No. Observations:                1504   AIC:                             7208.
Df Residuals:                    1325   BIC:                             8160.
Df Model:                         178                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

# Problem 4 - Fitting the LASSO Model {-}

Our third model is the LASSO, which can be implemented with the [LassoCV](https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LassoCV.html) function in the `sklearn` library. Run the following code to import this function.

```python
from sklearn.linear_model import LassoCV
```

And if needed, you can install `sklearn` by running the following in a separate cell.

```python
pip install scikit-learn
```


As is common in the machine learning literature, the syntax for applying the LASSO model does not expect a formula notation. The `LassoCV` function takes as arguments model matrices of the type described in parts 3-A and 3-B above. Importantly, these functions also do __not__ require us to explicitly create an intercept column in our data, as this will be included by default.

Problem 4-A below asks you to create these matrics.

## 4-A: Training and Testing X Matrices {-}

Create two new matrices, named `X_train` and `X_test`. These matrices should take the data you created from your training and testing splits respectively, but should exclude all the variables which were excluded from the long-regression. These matrices should have 178 columns. 

The `X_train` matrix will be used for training the model.
The `X_test` matrix can be used to predict our model on new data.

In [241]:
# Creating the numeric matrices for the data

X_train = osha_train.drop(columns=cols_excluded)
X_test = osha_test.drop(columns=cols_excluded)

## 4-B Fitting the LASSO Model {-}

Once you have created the matrices in this way, fit a LASSO model with the following code. To make your answer consistent with the solutions be sure to set the random_state to 407.

```python
mod_lasso = LassoCV(cv=10, random_state=407, max_iter=10**4).fit(X_train, y_c_train)
```
The `cv=10` argument in the above function call instructs the LASSO to be fit with 10-fold cross-validation to estimate the best value for the $\lambda$ penalty parameter.

In [242]:
# Fit your LASSO Model
from sklearn.linear_model import LassoCV

mod_lasso = LassoCV(cv=10, random_state=407, max_iter=10**4).fit(X_train, y_c_train)

## 4-C: Extracting Values From the LASSO Model {-}

There are many different values in the `mod_lasso` object created above. You can access the individual coefficient estimates from the model with the following code:

```python
mod_lasso.coef_
```

Running this code will produce an array of the individual estimated coefficient values across all the variables used for prediction (sometimes referred to as features). Using this array, report the following calculations.

1. How many coefficients estimated in the LASSO model are non-zero?
2. What percentage of coefficients estimated in the LASSO model are non-zero?

Hint: various `numpy` or other functions can be applied to the array output, including Boolean operations (>, ==, <, etc.). 

In [243]:
# Non-zero coefficient count / mean
count = np.sum(mod_lasso.coef_ != 0)
print(f'{count} coefficients in the LASSO model are non-zero.')

tot_count = len(mod_lasso.coef_)
nonzero_percentage = count/tot_count * 100
print(f'{nonzero_percentage:.2f}% of the coefficients estimated in the LASSO model are non-zero.')

28 coefficients in the LASSO model are non-zero.
15.73% of the coefficients estimated in the LASSO model are non-zero.


How do your above calculations compare to the initial linear regressions you ran? If you have fit a linear model, the `.params` attribute can be used to extract the estimated beta coefficients.

For both the short and long regression models, report what percentage of the estimated coefficients are non-zero.

In [244]:
# Your calculations here:

short_coef = short_reg.params
tot_count_short = len(short_coef)
nonzero_short = np.sum(short_coef != 0)
nz_perc_short = nonzero_short/tot_count_short * 100

long_coef = long_reg.params
tot_count_long = len(long_coef)
nonzero_long = np.sum(long_coef != 0)
nz_perc_long = nonzero_long/tot_count_long * 100

print(f'''The percentage of the estimated coefficients that are non-zero in the regressions are:
      - Short: {nz_perc_short:.2f}%
      - Long: {nz_perc_long:.2f}%''')

The percentage of the estimated coefficients that are non-zero in the regressions are:
      - Short: 100.00%
      - Long: 100.00%


# Problem 5 - Evaluating Performance {-}

## 5-A: MSE In the Training Sample {-}

Summarize the predictive performance for each model (on the training set) by computing the Mean Squared Error (MSE) between the predicted value and the observed outcome, and enter these values into the first column of the below table in 5-C.

When filling in the table, please round your numbers to 2 digits.

You may calculate the MSE by hand using formulas presented in class (as shown with code in Problem Set 6) or by defining a function yourself. Alternatively, you may find it convenient to load in the `mean_squared_error` function from the `sklearn` package with the following code. You can read the documentation for this function [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html).

```python
from sklearn.metrics import mean_squared_error
```

Note: Problem 6 Set demonstrated how to calculate and get predictions for linear models fit from the `statsmodels` library. For the LASSO model, your model object will contain a method `.predict()` which will estimate predicted values applying the model to new data. To get LASSO predictions on the training data, you can apply this prediction method to the training data itself, using:

```python
mod_lasso.predict(X_train)
```

Similarly, to obtain predictions on the testing data, you can update the above code so that the new data you are predicting to is the testing data.

```python
mod_lasso.predict(X_test)
```

In [245]:
from sklearn.metrics import mean_squared_error

# Calculating MSE in the training sample for the short-regression

y_short_train = short_reg.predict(osha_train)
mse_short_train = mean_squared_error(y_c_train, y_short_train)
print(f"MSE short: {mse_short_train:.2f}")

MSE short: 7.08


In [246]:
# Calculating MSE in the training sample for the long-regression

y_long_train = long_reg.predict(osha_train)
mse_long_train = mean_squared_error(y_c_train, y_long_train)
print(f"MSE long: {mse_long_train:.2f}")

MSE long: 5.57


In [247]:
# Calculating MSE in the training sample for the LASSO regression

y_lasso_train = mod_lasso.predict(X_train)
mse_lasso_train = mean_squared_error(y_c_train, y_lasso_train)
print(f"MSE LASSO: {mse_lasso_train:.2f}")

MSE LASSO: 6.32


## 5-B: MSE In the Testing (Hold-out) Sample {-}

Now repeat the procedures in 5-A, but this time use your fitted models to predict for the test (hold-out) set. Note that you will need to use the equivalent of `X_train` but for the test set `X_test`. Report your final results by entering the test-set MSE in the second column of Table 5-C. Remember you will be using the parameters you estimated using observations in the training set to make predictions about the observations in the test set.

When filling in the table, please round your numbers to 2 digits.

In [248]:
# Calculating MSE in the holdout sample 

# Short regression
y_short_test = short_reg.predict(osha_test)
mse_short_test = mean_squared_error(y_c_test, y_short_test)
print(f"MSE short: {mse_short_test:.2f}")

# Long regression
y_long_test = long_reg.predict(osha_test)
mse_long_test = mean_squared_error(y_c_test, y_long_test)
print(f"MSE long: {mse_long_test:.2f}")

# LASSO
y_lasso_test = mod_lasso.predict(X_test)
mse_lasso_test = mean_squared_error(y_c_test, y_lasso_test)
print(f"MSE LASSO: {mse_lasso_test:.2f}")

MSE short: 7.99
MSE long: 7.78
MSE LASSO: 7.28


## 5-C: Summary Table {-}

|    Model      | In-Sample MSE | Hold-out MSE |
|----------     | ------------- | ------------ |
| Short         |     7.08      |     7.99     |
| Long          |     5.57      |     7.78     |
| LASSO         |     6.32      |     7.28     |


## 5-D: Your Conclusions {-}

What are three key conclusions you draw from the above Table regarding your ability to predict injury rates (1-2 paragraphs)?

_Your conclusions here:_

First, we can conclude that `the LASSO and the Long regressions` are the two best alternatives, as the Long's in-sample MSE is the lowest (5.57), and the LASSO's hold-out MSE is the lowest as well (7.28), especially compared to the Short one.

The Long model shows overfitting: while it performs better in-sample, its hold-out MSE is actually worse than the LASSO and only marginally better than the Short model. This suggests that including all 178 predictors improves fit to the training data but does not improve predictive power on unseen data, and might introduce noise.
The Short regression exhibits the weakest predictive performance overall.

The LASSO model has a better balance between the two, improving from the other two models, also avoiding overfitting.

# Problem 6 - Estimating Binary Outcomes {-}

In the previous sections, you predicted a continuous variable (`injury_rate`). The next set of questions are about predicting a binary variable (`high_injury_rate`). Rather than aim to estimate specific injury rates at all locations, this type of binary model focuses on classification of a site as having a high injury rate, but not estimating the actual injury rate.

This problem will walk you through code to generate predictions from a Logistic Regression. The focus here will not be on the exact code of the Logistic Regression, but you will use predictions from these models to calculate and report precision and recall.

## 6-A: Fitting Logistic Models {-}

A logistic regression can be fit using functionality from `sklearn` by loading the following function.

```python
from sklearn.linear_model import LogisticRegression
```

Like the LASSO regression, this model will expect as inputs matrix representations of the data. The following blocks of code create these matrices (for the short regression, we can use the same matrices generated earlier for the long regression) and also save down the outcome variables we are interested in. We can use the same training / testing split from above.

Run this code before moving on to the next sections.

In [249]:
from sklearn.linear_model import LogisticRegression

# Saving down the new outcome variables
y_b_train = osha_train['high_injury_rate']
y_b_test = osha_test['high_injury_rate']

# Creating a matrix format for the short regression, both the training and testing versions
X_train_short = osha_train.loc[:,['has_tmin1_odi', 'any_insp_prior', 'any_complaint_tmin13', 'num_nonfat_comp_insp_cy_tc99mm1',
                        'initial_pen_cy_mzmm1', 'ln_initial_pen_cy_mzmm1', 'dafw_analysis_rec_tc99mm1']]

X_test_short = osha_test.loc[:,['has_tmin1_odi', 'any_insp_prior', 'any_complaint_tmin13', 'num_nonfat_comp_insp_cy_tc99mm1',
                        'initial_pen_cy_mzmm1', 'ln_initial_pen_cy_mzmm1', 'dafw_analysis_rec_tc99mm1']]

## 6-B: Fitting Basic Logistic Regressions {-}

The following code fits basic logistic regressions for both the long and short regressions, and a logistic regression with cross-validation and a LASSO penalty. Run this code before moving on to the next exercises.

Note, the LASSO regression may take a few minutes to fit (depending on your computer).

In [250]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

# Fitting the short model
fit_short_b = LogisticRegression(max_iter=10**3).fit(X_train_short, y_b_train)

# Fitting the long model
fit_long_b = LogisticRegression(max_iter=10**3).fit(X_train, y_b_train)

# Fitting a model with a LASSO penalty
fit_lasso_b = LogisticRegressionCV(cv=5, penalty='l1', solver='liblinear', random_state=0, max_iter=10**4).fit(X_train, y_b_train)

## 6-C: Extracting Predicted Probabilities {-}

The models you just fit can be used to generate predicted probabilities of whether a site will have a value of `1` for the `high_injury_rate` variable or a `0`. The predicted probabilities of having `high_injury_rate` can be calculated and extracted with the `.predict_proba()` method. The below code extracts and saves these probabilities for all three models (short, long, LASSO) for the training data. This information will be needed later to calculate precision and recall. Run this code, and then modify it as needed to similarly save predicted probabilities for the testing data.

In [251]:
# TRAINING DATA PREDICTIONS

# Predictions from the short regression
p_short_train = fit_short_b.predict_proba(X_train_short)[:,1]
# Predictions from the long regression
p_long_train = fit_long_b.predict_proba(X_train)[:,1]
# Predictions from the LASSO regression
p_lasso_train = fit_lasso_b.predict_proba(X_train)[:,1]

In [252]:
# TESTING DATA PREDICTIONS

# Your code here:

# Short regression
p_short_test = fit_short_b.predict_proba(X_test_short)[:,1]
# Long regression
p_long_test = fit_long_b.predict_proba(X_test)[:,1]
# LASSO regression
p_lasso_test = fit_lasso_b.predict_proba(X_test)[:,1]

You can work with these arrays of predicted probabilities individually or you may prefer to save them together in a single Data Frame along with the true outcomes `y_b_train` and `y_b_test`. You may use the space below to create this DataFrame. 

In [253]:
# Store the predictions in a DataFrame

df_training = pd.DataFrame(
    {
        'Short Train Probabilities': p_short_train,
        'Long Train Probabilities': p_long_train,
        'LASSO Train Probabilities': p_lasso_train
    }
)

df_testing = pd.DataFrame(
    {
        'Short Test Probabilities': p_short_test,
        'Long Test Probabilities': p_long_test,
        'LASSO Test Probabilities': p_lasso_test
    }
)

## 6-D: Estimating Binary Outcomes {-}

The above predicted probabilities can be used to identify high-risk sites. The default behavior for the `sklearn` logistic models is to predict the binary outcome variable to be a `1` wherever this predicted probability exceeds 50%. Given the limited budget of OSHA however we may not be able to go to all sites. Further, OSHA may also wish to go to a site where the predicted probability is below 50% if that site is still one of the highest predicted probabilities in our data set. Given the budget constraint, we could instead focus on the __top 30%__ of the sites with the highest predicted probabilities within each category.

A pandas DataFrame has a `.quantile()` method, or the numpy library has the `numpy.quantile()` function. Using these functions (or another approach you are comfortable with), for each of the three models (short, long, LASSO) and the two data sets (training and testing), create new variables which store the _binary predictions_ (values of `0` or `1`) for the `high_injury_rate` variable where the top 30% of predicted probabilities __within each model__ are estimated to a value of `1` (denoting high injury) and the bottom 70% of predicted probabilities __within each model__ are estimated to a value of `0`.

At the end of this process, you should have 3 different binary estimates (one for each of the short, long and LASSO models) for each observation across the two different samples (training and testing).

The quantile calculation should be done within each of these six separate categories (e.g. for your binary predictions on the training data for the short regression model, you should examine the top 30% of predicted probabilities for the short regression model in the training data).

Hint: you can use a Boolean operator (<, >, ==, etc.) to compare all values in a specific column or array to a single calculated value (such as the desired quantile thresholds).

In [254]:
# Your code here:

for col in df_training.columns:
    threshold_train = df_training[col].quantile(.7)
    df_training[f"{col} Binary"] = (df_training[col] >= threshold_train).astype(int)

for col in df_testing.columns:
    threshold_test = df_testing[col].quantile(.7)
    df_testing[f"{col} Binary"] = (df_testing[col] >= threshold_test).astype(int)

## 6-E: Calculating Precision and Recall {-}

While MSE is a common measure of model accuracy for continuous outcomes, when working with binary outcomes in a classification problem, _precision_ and _recall_ are more popular measures. These measures were defined in the case study and in class.

With your above calculations, you have the true values for the outcome (which were stored as `y_b_train` and `y_b_test` when fitting the models), and the corresponding predictions based on the 30% cutoff.

Using these values, calculate and report the precision and recall for each model both within the training sample and the testing sample and fill in the below table.

You may calculate these values using the formulas provided in the case study. Alternatively you may wish to use functions provided in the `sklearn` package which can be loaded with the following code:

```python
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
```

You may read the documentation for these functions at: [recall_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html) and [precision_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html).

In [255]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# Your code here:

# TRAINING

    # Short
y_pred_short_train = df_training['Short Train Probabilities Binary']
precision_train_short = precision_score(y_b_train, y_pred_short_train)
recall_train_short = recall_score(y_b_train, y_pred_short_train)

    # Long
y_pred_long_train = df_training['Long Train Probabilities Binary']
precision_train_long = precision_score(y_b_train, y_pred_long_train)
recall_train_long = recall_score(y_b_train, y_pred_long_train)

    # LASSO
y_pred_lasso_train = df_training['LASSO Train Probabilities Binary']
precision_train_lasso = precision_score(y_b_train, y_pred_lasso_train)
recall_train_lasso = recall_score(y_b_train, y_pred_lasso_train)

print(
    f'''
    Training dataframe
    - Short precision and recall: {precision_train_short:.2f}, {recall_train_short:.2f};
    - Long precision and recall: {precision_train_long:.2f}, {recall_train_long:.2f};
    - LASSO precision and recall: {precision_train_lasso:.2f}, {recall_train_lasso:.2f}.
    '''
)


# TESTING

    # Short
y_pred_short_test = df_testing['Short Test Probabilities Binary']
precision_test_short = precision_score(y_b_test, y_pred_short_test)
recall_test_short = recall_score(y_b_test, y_pred_short_test)

    # Long
y_pred_long_test = df_testing['Long Test Probabilities Binary']
precision_test_long = precision_score(y_b_test, y_pred_long_test)
recall_test_long = recall_score(y_b_test, y_pred_long_test)

    # LASSO
y_pred_lasso_test = df_testing['LASSO Test Probabilities Binary']
precision_test_lasso = precision_score(y_b_test, y_pred_lasso_test)
recall_test_lasso = recall_score(y_b_test, y_pred_lasso_test)

print(
    f'''
    Testing dataframe
    - Short precision and recall: {precision_test_short:.2f}, {recall_test_short:.2f};
    - Long precision and recall: {precision_test_long:.2f}, {recall_test_long:.2f};
    - LASSO precision and recall: {precision_test_lasso:.2f}, {recall_test_lasso:.2f}.
    '''
)


    Training dataframe
    - Short precision and recall: 0.75, 0.51;
    - Long precision and recall: 0.84, 0.57;
    - LASSO precision and recall: 0.78, 0.53.
    

    Testing dataframe
    - Short precision and recall: 0.76, 0.53;
    - Long precision and recall: 0.76, 0.53;
    - LASSO precision and recall: 0.77, 0.54.
    


|    Model      | Training Precision | Training Recall |
|----------     | -------------      | ------------    |
| Short         |    0.75                |     0.51            |
| Long          |        0.84            |       0.57          |
| LASSO         |            0.78        |           0.53      |

|    Model      | Testing Precision | Testing Recall |
|----------     | -------------       | ------------     |
| Short         | 0.76         |       0.53          |
| Long          |      0.76              |     0.53             |
| LASSO         |          0.77          |     0.54            |

# Problem 7 - Final Thoughts {-}

## 7-A: Selecting an Approach {-}

Based on the results of your tables in Problem 5 and Problem 6, OSHA asks you to indicate which of your prediction algorithms you would recommend using to select the sites they should inspect and why (2-3 paragraphs). 

_Your selection here:_

My suggestion would be to use the `LASSO model`: although all three prediction methods perform similarly on the test sample, LASSO achieves the strongest balance between accuracy and generalizability. In particular, it attains the highest testing precision (0.77) and recall (0.54), indicating that it is slightly more effective at identifying establishments that truly face a high injury risk. This means OSHA would be better able to target inspections toward the workplaces that most warrant regulatory attention, while reducing the number of unnecessary inspections of low-risk sites.

Compared to the Long regression model — the alternative closest to it — the advantages of LASSO become clear, as the Long model exhibits signs of overfitting, performing very well on the training data but losing much of that performance on the test set. This suggests that it captures noise rather than stable, predictive patterns. By contrast, the LASSO penalty helps control model complexity and improves its out-of-sample stability. As a result, LASSO provides more reliable predictions for new data, making it a more suitable and operationally robust choice for OSHA’s inspection strategy.

## 7-B: OSHA's Current Approach {-}

Given OSHA’s current approach of choosing randomly to inspect 30% of sites, __compute the precision and recall of their current approach__. Note: You will not need to use python or the data set to answer this question. The goal is that you apply the definition of precision and recall to OSHA’s current algorithm.

From the full data, the proportion of sites in the full sample that have a high injury rate is 44%. Adding notation, we can define the above facts as:

- Pr(High Injury) = 0.44
- Pr(Selected) = 0.30

The current approach of random selection assumes these two probabilities are independent.


_Your calculation here:_

$Recall=Pr(Selected|High\ \ Injury)=Pr(S \cap\ H)/Pr(H)=(0.30*0.44)/0.44=0.30$

$Precision=Pr(High\ \ Injury|Selected)=Pr(H \cap\ S)/Pr(S)=(0.44*0.30)/0.30=0.44$

## 7-C: Comparing your Precision and Recall Rates {-}

Briefly compare precision and recall from your models compared to OSHA's current approach (1 paragraph).


_Your answer here:_

The values calculated – assuming that the probabilities are independent – are:
- `Recall = 0.30`
- `Precision = 0.44`

These are significantly lower than the ones obtained in the Training and Testing calculations, especially compared to the LASSO values, closer to (but still lower than) the Short regression model's results.

Compared to OSHA’s current random-selection approach, the predictive models all perform meaningfully better. The random method cannot exploit any information in the data, leading to low accuracy in identifying truly high-risk sites. In contrast, the LASSO model achieves substantially higher recall and precision, indicating that it is far more effective at detecting establishments with elevated injury rates while reducing false positives. Even the Short regression model, although less accurate than LASSO, surpasses the performance of the current approach. Overall, this comparison shows that OSHA’s existing method is considerably less efficient than the data-driven predictive alternative.

## 7-D: Additional Considerations {-}

OSHA would need to overcome several operational and political obstacles in order to change their algorithm for targeting which sites to inspect. They ask you to explain to them how much better the algorithm you chose performs relative to their status quo operation. How would you respond? [1 paragraph]

_Your explanation here:_

The difference between the actual random method and the (superior) LASSO approach is that random selection cannot leverage predictive patterns in the data. Adopting a predictive algorithm like LASSO would lead to more efficient resource allocation and better identification of high-risk workplaces.

The LASSO model, operating with the same inspection budget, offers superior performance with a Recall of 0.54 and Precision of 0.77. Compared to random selection (which would achieve approximately 0.30 recall and 0.44 precision), this represents a substantial improvement in both metrics, meaning OSHA could inspect the same number of sites while identifying significantly more high-risk workplaces and reducing false positives.

# Appendix 0 - AI Promts {-}

If you used GenAI assistance in completing this problem set, please include the prompts you used below.

#### 4-C:
_Are these answers reliable, even if all at 100%?_

(From my results) _The percentage of the estimated coefficients that are non-zero in the regressions are:_

   _- Short: 100%_

   _- Long: 100%_

#### 6-B:

(Before fixing my code, I had troubles with running the command) _The command below does not work, is it okay if I increase the number of iterations?_

_Command:_

_fit_short_b = LogisticRegression(max_iter=10**3).fit(X_train_short, y_b_train)_

_fit_long_b = LogisticRegression(max_iter=10**3).fit(X_train, y_b_train)_

_(...)_

As a result, I increased the iterations of the long model. After fixing the code – due to a mistake – I brought the iterations number back to its initial amount.

In [1]:
!jupyter nbconvert --to html "63387_PP422.ipynb"

[NbConvertApp] Converting notebook 63387_PP422.ipynb to html
[NbConvertApp] Writing 461065 bytes to 63387_PP422.html
